In [6]:
import cv2 as cv
print(cv.__version__)

import numpy as np
print(np.__version__)

import matplotlib.pyplot as plt

import time

4.10.0
1.26.4


In [7]:
class ImagePipeline:
    def __init__(self, enable_face_detect=False, tint_mode=None):
        """
        初始化 pipeline
        :param enable_face_detect: 是否启用人脸检测
        :param tint_mode: 色调滤镜模式，可选 'warm', 'cool', None
        """
        self.enable_face_detect = enable_face_detect
        self.tint_mode = tint_mode
        if self.enable_face_detect:
            try:
                self.face_cascade = cv.CascadeClassifier(
                    cv.data.haarcascades + 'haarcascade_frontalface_default.xml'
                )
            except AttributeError:
                print("⚠️ cv.data 不可用，跳过人脸检测")
                self.enable_face_detect = False

    # ========== 原有方法（略作保留）==========
    def flip_horizontal(self, frame):
        return cv.flip(frame, 1)

    def adjust_brightness_hsv(self, frame, value_delta=30):
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv)
        v = np.clip(v.astype(np.int16) + value_delta, 0, 255).astype(np.uint8)
        return cv.cvtColor(cv.merge([h, s, v]), cv.COLOR_HSV2BGR)

    def detect_faces(self, frame):
        if not self.enable_face_detect:
            return frame
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        return frame

    # ========== 新增方法 ==========
    
    def bilateral_filter(self, frame, d=9, sigma_color=75, sigma_space=75):
        """
        双边滤波：平滑图像但保留边缘（美颜去噪）
        :param d: 邻域直径（建议 5~15）
        :param sigma_color: 色彩空间标准差
        :param sigma_space: 坐标空间标准差
        """
        return cv.bilateralFilter(frame, d, sigma_color, sigma_space)

    def white_balance_grayworld(self, frame):
        """
        灰度世界白平衡算法（简单有效）
        假设图像平均 R=G=B，通过缩放通道实现平衡
        """
        if frame.dtype != np.uint8:
            frame = frame.astype(np.uint8)
        result = frame.copy()
        # 计算各通道均值
        avg_b = np.mean(frame[:, :, 0])
        avg_g = np.mean(frame[:, :, 1])
        avg_r = np.mean(frame[:, :, 2])

        # 避免除零
        if avg_b == 0 or avg_g == 0 or avg_r == 0:
            return frame

        # 计算缩放因子（以绿色通道为基准）
        scale_b = avg_g / avg_b
        scale_r = avg_g / avg_r

        result[:, :, 0] = np.clip(frame[:, :, 0] * scale_b, 0, 255)
        result[:, :, 2] = np.clip(frame[:, :, 2] * scale_r, 0, 255)
        return result.astype(np.uint8)

    def apply_tint(self, frame):
        """
        应用色调滤镜：
        - warm: 增加红色/黄色（模拟夕阳）
        - cool: 增加蓝色（模拟阴天）
        """
        if self.tint_mode is None:
            return frame

        overlay = frame.copy()
        output = frame.copy()

        if self.tint_mode == 'warm':
            # 暖色：添加橙色覆盖层
            overlay[:] = (40, 80, 200)  # BGR: 偏红黄
            alpha = 0.2
        elif self.tint_mode == 'cool':
            # 冷色：添加蓝色覆盖层
            overlay[:] = (200, 120, 40)  # BGR: 偏蓝青
            alpha = 0.15
        else:
            return frame

        # 混合原图与覆盖层
        return cv.addWeighted(overlay, alpha, output, 1 - alpha, 0)

    # ========== 主处理流水线 ==========
    def process(self, frame):
        if frame is None or frame.size == 0:
            return frame

        out = frame.copy()

        # Step 1: 白平衡（建议早做）
        out = self.white_balance_grayworld(out)

        # Step 2: 镜像
        out = self.flip_horizontal(out)

        # Step 3: 亮度增强
        out = self.adjust_brightness_hsv(out, value_delta=35)

        # Step 4: 双边滤波（美颜去噪）
        out = self.bilateral_filter(out, d=7, sigma_color=50, sigma_space=50)

        # Step 5: 色调滤镜
        out = self.apply_tint(out)

        # Step 6: 人脸检测（最后叠加图形）
        out = self.detect_faces(out)

        return out

In [3]:
# ======================
# 主程序：运行摄像头 pipeline
# ======================
def main():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("❌ 无法打开摄像头")
        return

    # 可尝试: tint_mode=None, 'warm', 'cool'
    pipeline = ImagePipeline(enable_face_detect=True, tint_mode='cool')

    print("按 'q' 退出")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        processed = pipeline.process(frame)
        combined = np.hstack((frame, processed))
        cv.imshow('Original | Processed', combined)

        key = cv.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

In [10]:
if __name__ == "__main__":
    main()

⚠️ cv.data 不可用，跳过人脸检测
📷 摄像头启动中... 按 'q' 退出
